In [ ]:
import numpy as np
import cv2
import cv2.aruco as aruco
import glob

marker_length = 0.015  # Length of each marker's side (in meters or millimeters)

obj_points = []  # List to store object points for each image
img_points = []  # List to store image points for each image

aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)  
board = aruco.GridBoard((8, 11), marker_length, 0.011, aruco_dict)  
parameters =  cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)

# Get the list of images containing the ArUco board
images = glob.glob('./callibration_images/*.jpg')  

# Loop through each image
for image_path in images:
    # Read the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

    # Detect markers in the image
    corners, ids, rejectedImgPoints = detector.detectMarkers(gray)

    # If markers are detected, proceed with the calibration process
    if len(corners) > 0:
        # Add object points (same for each image as the board is fixed)
        obj_points.append(board.getObjPoints()[0])  # Add object points for the grid
        img_points.append(corners[0])  # Add detected corners from the image

        # Draw detected markers on the image (optional visualization)
        img_markers = aruco.drawDetectedMarkers(img.copy(), corners, ids)
        cv2.imshow('Detected Markers', img_markers)
        cv2.waitKey(500)  # Wait 500 ms before showing the next image

# Close any open OpenCV windows
cv2.destroyAllWindows()

# Perform camera calibration if sufficient images are found
if len(obj_points) > 0:
    # Calibrate the camera using the collected object points and image points
    ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)

    if ret:
        print("Camera calibration successful!")
        print("Camera matrix:\n", camera_matrix)
        print("Distortion coefficients:\n", dist_coeffs)
    else:
        print("Calibration failed!")
else:
    print("No valid images detected for calibration.")


Camera calibration successful!
Camera matrix:
 [[1.83921948e+03 0.00000000e+00 3.18746645e+02]
 [0.00000000e+00 1.39871140e+03 2.42980391e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients:
 [[ 9.14134180e-01  8.71723947e+01  1.72328813e-01  2.09509326e-01
  -1.99624787e+03]]


In [16]:
# Undistort an image using the calibration results
img = cv2.imread('./callibration_images/image_0.jpg')
h, w = img.shape[:2]
new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, dist_coeffs, (w, h), 1, (w, h))

# Undistort the image
undistorted_img = cv2.undistort(img, camera_matrix, dist_coeffs, None, new_camera_matrix)

# Crop the image based on the region of interest (ROI)
x, y, w, h = roi
undistorted_img = undistorted_img[y:y+h, x:x+w]

# Show the undistorted image
cv2.imshow('Undistorted Image', undistorted_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
